In [47]:
%matplotlib inline 
from __future__ import division
from pylab import *
import skimage as ski
from skimage import data, io, filters, exposure, measure
from skimage.filters import rank
from skimage import img_as_float, img_as_ubyte
from skimage.morphology import disk
import skimage.morphology as mp
from skimage import util
from skimage.color import rgb2hsv, hsv2rgb, rgb2gray, gray2rgb
from skimage.filters.edges import convolve
from matplotlib import pylab as plt
import numpy as np
from numpy import array
from IPython.display import display
from ipywidgets import interact, interactive, fixed
from ipywidgets import *
from ipykernel.pylab.backend_inline import flush_figures
from multiprocessing.pool import ThreadPool
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.draw import circle_perimeter

In [71]:
dicesToRead = ['01']
def drawDiceImage(i, img):
    plt.subplot(1,2,i)
    plt.imshow(img)

In [72]:
dices = [io.imread('./dices/dice{0}.jpg'.format(i)) for i in dicesToRead]

In [74]:
def getEdges(img, gamma = 0.7, sig=3, l=0, u=100): #gama 0.7 sig = 4.65
    img = rgb2gray(img)
#     pp,pk = np.percentile(img,(l,u));
#     img = exposure.rescale_intensity(img,in_range=(pp,pk))
    from skimage import feature
#     img = img ** gamma
    img = ski.feature.canny(img, sigma=sig)
#     img = mp.dilation(img)
    return img

In [82]:
def findCircles(image, edges):
    # Detect two radii
    hough_radii = np.arange(5, 15, 4)
    hough_res = hough_circle(edges, hough_radii)

    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                               total_num_peaks=10)

    # Draw them
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    image = gray2rgb(image)
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        image[circy, circx] = (220, 20, 20)
    return image

In [83]:
def drawDices(gamma=1.4, sig = 1.7, l=0, u=100):
    fig = plt.figure(facecolor="black", figsize=(60,60))
    i = 1
    for img in dices:
        edges = getEdges(img, gamma, sig, l, u)
        img = findCircles(img, edges)
        drawDiceImage(i, edges)
        i +=1 
        drawDiceImage(i, img)
    plt.tight_layout()
    plt.show()
    fig.savefig("dices.pdf",facecolor="black")
    plt.close()

interact(drawDices, gamma=(0.1,2,0.1), sig=(0.1,4,0.1), l=(0,100,1), u = (0,100,1))

interactive(children=(FloatSlider(value=1.4, description='gamma', max=2.0, min=0.1), FloatSlider(value=1.7, description='sig', max=4.0, min=0.1), IntSlider(value=0, description='l'), IntSlider(value=100, description='u'), Output()), _dom_classes=('widget-interact',))

<function __main__.drawDices>